# Bundle adjustment of $n$-cameras

### Goal

Suppose $n$ cameras have been calibrated linearly beforehand, using 2D-2D correspondences for example.  We assume we have an initial guess of $R$ and $t$ for each camera satisfying $x \sim A (R X + t)$, the 2D observation $x$, and the 3D point $X$.

* The initial guess is not necessarily done by linear methods.  It can even be done by hand.
* The 3D point $X$ can be triangulated from $x$ using the initial $R, t$.
* The $n$ cameras do not necessarily observe all the points.

Given $A, d, R, t, x, X$, this notebook optimizes $A, d, R, t, X$ so as to minimize the reprojection error.

* Input:
  * initial guess of $A, d, R, t, X$ and 2D observations $x$
* Output:
  * optimal $A, d, R, t, X$
 

## Libraries

In [1]:
%matplotlib notebook
import sys, os, cv2
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from pycalib.plot import plotCamera
from pycalib.ba import bundle_adjustment, encode_camera_param, decode_camera_param, make_mask
from pycalib.calib import lookat, triangulate

np.random.seed(0)

## Synthetic data

In [2]:
# 3D points
# X_gt = (np.random.rand(16, 3) - 0.5)*5 # random points centered at [0, 0, 0]
X_gt = np.array(np.meshgrid(np.linspace(-1, 1, 3), np.linspace(-1, 1, 3), np.linspace(-1, 1, 3))).reshape((3, -1)).T  # 3D grid points
Np = X_gt.shape[0]
print('X_gt:', X_gt.shape)

# Camera intrinsics
K = np.array([[600, 0, 320], [0, 600, 240], [0, 0, 1]]).astype(np.float64)  # VGA camera

# Camera poses: cameras are at the vertices of a hexagon
t = 2 * np.pi / 5 * np.arange(5)
v_gt = np.vstack((10*np.cos(t), 10*np.sin(t), np.zeros(t.shape))).T
Nc = v_gt.shape[0]
R_gt = []
t_gt = []
P_gt = []
rvec_gt = []
for i in range(Nc):
    t = v_gt[i,:]
    R, t = lookat(t, np.zeros(3), np.array([0, 1, 0]))
    R_gt.append(R)
    t_gt.append(t)
    P_gt.append(K @ np.hstack((R, t)))
    rvec_gt.append(cv2.Rodrigues(R)[0])
R_gt = np.array(R_gt)
t_gt = np.array(t_gt)
P_gt = np.array(P_gt)
rvec_gt = np.array(rvec_gt)
print('R_gt:', R_gt.shape)
print('t_gt:', t_gt.shape)
print('P_gt:', P_gt.shape)
print('rvec_gt:', rvec_gt.shape)

# 2D observations points
x_gt = []
for i in range(Nc):
    xt = cv2.projectPoints(X_gt.reshape((-1, 1, 3)), rvec_gt[i], t_gt[i], K, None)[0].reshape((-1, 2))
    x_gt.append(xt)
x_gt = np.array(x_gt)
print('x_gt:', x_gt.shape)

# Verify triangulation
Y = []
for i in range(Np):
    y = triangulate(x_gt[:,i,:].reshape((-1,2)), P_gt)
    #print(y)
    Y.append(y)
Y = np.array(Y).T
Y = Y[:3,:] / Y[3,:]
assert np.allclose(0, X_gt - Y.T)

# Verify z > 0 at each camera
for i in range(Nc):
    Xc = R_gt[i] @ X_gt.T + t_gt[i]
    assert np.all(Xc[2, :] > 0)

    
# Inject gaussian noise to the inital guess
R_est = R_gt.copy()
t_est = t_gt.copy()
K_est = np.array([K for c in range(Nc)])
X_est = X_gt.copy()
x_est = x_gt.copy()

for i in range(Nc):
    R_est[i] = cv2.Rodrigues( cv2.Rodrigues(R_est[i])[0] + np.random.normal(0, 0.01, (3,1)) )[0]
    t_est[i] += np.random.normal(0, 0.01, (3,1))
    K_est[i][0,0] = K_est[i][1,1] = K_est[i][0,0] + np.random.normal(0, K_est[i][0,0]/10)

X_est += np.random.normal(0, 0.01, X_est.shape)
x_est += np.random.normal(0, 0.1, x_est.shape)

X_gt: (27, 3)
R_gt: (5, 3, 3)
t_gt: (5, 3, 1)
P_gt: (5, 3, 4)
rvec_gt: (5, 3, 1)
x_gt: (5, 27, 2)


## Bundle adjustment

In [3]:
# def bundle_adjustment(camera_params, points_3d, camera_indices, point_indices, points_2d, *, verbose=2, mask=None):

# Camera parameters
camera_params = []
for i in range(Nc):
    c = encode_camera_param(R_est[i], t_est[i], K_est[i], np.zeros(5))
    camera_params.append(c)
camera_params = np.array(camera_params)

# camera_indices[i] == the camera observes point_2d[i,:]
camera_indices = np.repeat(np.arange(Nc), Np)

# point_indices[i] == the 3D point behind point_2d[i,:]
point_indices = np.tile(np.arange(Np), Nc)

# Optimization target: 1D mask == all cameras share a same mask
# R, t, f, u0, v0, k1, k2, p1, p2, k3
mask = make_mask(True, True, f=True, u0=True, v0=True, k1=True, k2=True, p1=True, p2=True, k3=True, k4=False, k5=False, k6=False)

# (Optional) 2D mask == Nc x mask == camera-wise mask
mask = np.tile(mask, (Nc, 1))
mask[1, 6:] = False  # fix the intrinsics of CAM1

# bundle_adjustment accepts both 1D and 2D masks
cam_opt, X_opt, e, ret = bundle_adjustment(camera_params, X_est, camera_indices, point_indices, x_est.reshape((-1, 2)), mask=mask)

print('reproj error = ', e)
print(X_gt)
print(X_opt)

for i in range(Nc):
    print(f'\nCamera {i}')
    print('before:')
    print(camera_params[i])
    print('after:')
    print(cam_opt[i])

assert e < 1.0


   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         5.1154e+03                                    2.97e+03    
       1              2         4.4594e+00      5.11e+03       1.01e+03       4.75e+02    
       2              3         2.7977e+00      1.66e+00       2.29e+03       5.46e+01    
       3              4         6.4105e-01      2.16e+00       9.79e+02       1.16e+01    
       4              5         5.7548e-01      6.56e-02       4.40e+02       4.63e+00    
       5              6         5.7144e-01      4.04e-03       4.68e+02       5.10e+00    
       6              7         5.7108e-01      3.58e-04       7.85e+02       2.76e+00    
       7              8         5.6727e-01      3.81e-03       1.62e+02       7.42e-01    
       8              9         5.6653e-01      7.38e-04       4.52e+01       2.69e-01    
       9             10         5.6537e-01      1.17e-03       6.63e+01       2.89e-01    